<a href="https://colab.research.google.com/github/sateesh12/tf/blob/main/vgg16_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import zipfile
import requests
import glob as glob

from tensorflow.keras.utils import image_dataset_from_directory
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass

block_plot = False
plt.rcParams['figure.figsize'] = (12,9)
SEED_VALUE = 42

In [ ]:
def download_and_unzip(url, save_name):
    #url = url
    file = requests.get(url)
    open(save_name, 'wb').write(file.content)
    try:
        with zipfile.ZipFile(save_name) as z:
            z.extractall("./")
            print('Extracted all')
    except:
        print('Invalid data')

download_and_unzip('https://www.dropbox.com/s/6nrjxr2ycnpcy63/dataset_balls.zip?dl=1',
    'dataset_balls.zip')

In [ ]:
from keras.api._v2.keras import datasets
@dataclass (frozen=True)
class DatasetConfig:
    NUM_CLASSES:  int = 10
    IMG_HEIGHT:    int = 224
    IMG_WIDTH:     int  = 224
    CHANNELS:        int  = 3
    BATCH_SIZE:     int  = 32
    DATA_ROOT_TRAIN: str='./dataset_balls/train'
    DATA_ROOT_VALID: str='./dataset_balls/valid'

@dataclass(frozen=True)
class TrainingConfig:
    BATCH_SIZE: int = 32
    EPOCSHS:     int = 51
    LEARNING_RATE: float = 0.0001
    CHECKPOINT_DIR: str = './saved_models_balls'


In [ ]:
# Load an existing VGG16  model using Keras.
input_shape = (DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH, DatasetConfig.CHANNELS)
print('Load the model with random weights')
vgg16_conv_base = tf.keras.applications.vgg16.VGG16(input_shape=input_shape, include_top=False, weights=None)
vgg16_conv_base.trainable = True
print('All kernel weights are trainable')
print(vgg16_conv_base.summary())

In [ ]:
# Add classifictaion layer, the above CNN extracted the features.
inputs = tf.keras.Input(shape=input_shape)

x = tf.keras.applications.vgg16.preprocess_input(inputs)
x = vgg16_conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(DatasetConfig.NUM_CLASSES, activation='softmax')(x)
vgg16_model = keras.Model(inputs, outputs)
print(vgg16_model.summary())

In [ ]:
# Split into training and validation set
train_dataset = image_dataset_from_directory(directory=DatasetConfig.DATA_ROOT_TRAIN,
                                                                        batch_size=TrainingConfig.BATCH_SIZE,
                                                                        seed = SEED_VALUE,
                                                                        label_mode='categorical',
                                                                        image_size = (DatasetConfig.IMG_WIDTH, DatasetConfig.IMG_HEIGHT))
valid_dataset = image_dataset_from_directory(directory=DatasetConfig.DATA_ROOT_VALID,
                                                                        batch_size=TrainingConfig.BATCH_SIZE,
                                                                        seed = SEED_VALUE,
                                                                        label_mode='categorical',
                                                                        image_size = (DatasetConfig.IMG_WIDTH, DatasetConfig.IMG_HEIGHT))

In [ ]:
from keras.engine.training import optimizer
# Compile and train the model of vgg-16
vgg16_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=TrainingConfig.LEARNING_RATE),
                                                       loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                                                       metrics=['accuracy'])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=TrainingConfig.CHECKPOINT_DIR,
                                                                                                save_weights_only=False,
                                                                                                monitor='val_accuracy',
                                                                                                mode='max',
                                                                                                save_best_only=True)
trainig_results = vgg16_model.fit(train_dataset,
                                                validation_data=valid_dataset,
                                                epochs=TrainingConfig.EPOCSHS,
                                                workers=4,
                                                use_multiprocessing=True,
                                                callbacks=model_checkpoint_callback)

In [ ]:
# Standard plotting function, should start to make a module of this !
def plot_results(metrics, ylabel=None, ylim=None, metric_name=None, color=None):

    fig, ax = plt.subplots(figsize=(15, 4))

    if not (isinstance(metric_name, list) or isinstance(metric_name, tuple)):
        metrics = [metrics,]
        metric_name = [metric_name,]

    for idx, metric in enumerate(metrics):
        ax.plot(metric, color=color[idx])

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(ylabel)
    plt.xlim([0, TrainingConfig.EPOCSHS-1])
    plt.ylim(ylim)
    # Tailor x-axis tick marks
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    plt.grid(True)
    plt.legend(metric_name)
    plt.show(block=block_plot)
    plt.close()

In [ ]:
train_loss = trainig_results.history['loss']
train_acc  = trainig_results.history['accuracy']
valid_loss = trainig_results.history['val_loss']
valid_acc  = trainig_results.history['val_accuracy']

max_loss = max(max(train_loss), max(valid_loss))
plot_results([train_acc, valid_acc], ylabel = 'Accuracy',
                ylim=[0.0, 1.0],
                metric_name = ["Training Accuracy"," Validation Accuracy"],
                color=["g","b"])

plot_results([train_loss, valid_loss], ylabel = 'Loss',
                ylim=[0.0, max_loss],
                metric_name = ["Training Loss", "Validation Loss"],
                color=["g","b"])

